In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from matplotlib.pylab import rcParams


In [5]:
df=pd.read_csv('/content/drive/My Drive/airline_passengers.csv')

In [6]:
df.columns=['date','passengers']

In [7]:
#dropping null values
df.dropna(inplace=True)

In [8]:
df['date']=pd.to_datetime(df['date'])

In [9]:
df.set_index('date',inplace=True)